In [ ]:
import os
import json
import pandas as pd

# 현재 디렉토리에 있는 모든 JSON 파일을 가져온 후 정렬
base_dir = 'Validation' # ['Trainig' or 'Validation']
if base_dir == 'Trainig':
    folder_path = 'Training/02.라벨링데이터/TL_1.일상트랜드_2.여행_관광_및_명소'
elif base_dir == 'Validation':
    folder_path = 'Validation/02.라벨링데이터/VL_1.일상트랜드_2.여행_관광_및_명소'
json_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.json')])


jeju_json_lt = []
data = []

# 각 JSON 파일에 대해 처리
for file_name in json_files:
    with open(folder_path+"/"+file_name, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

        # "keyword" 값 안에 "제주도"가 포함된 경우 처리
        # if "keyword" in json_data and
        if "제주" in json_data["info"]["keyword"]:
            # "speakerCId" 값이 null이 아닌 경우 넘어감
            if json_data['info']['speaker'].get("speakerCId") is not None:
                continue

            # "utterances"의 "speaker" 순서 체크 (speakerA, speakerB 순서 유지 확인)
            utterances = json_data.get('utterances', [])
            valid_sequence = True
            expected_speaker = "speakerA"

            for utterance in utterances:
                if utterance['speaker'] != expected_speaker:
                    valid_sequence = False
                    break
                expected_speaker = "speakerB" if expected_speaker == "speakerA" else "speakerA"

            if not valid_sequence:
                continue  # 순서가 맞지 않으면 다음 파일로 넘어감

            # speakerCId가 null이고 speaker 순서가 올바른 경우 데이터를 저장
            jeju_json_lt.append(file_name)
            info_id = json_data['info'].get('id', '')
            keyword = json_data['info'].get('keyword', '')

            for utterance in utterances:
                speaker = utterance.get('speaker', '')
                text = utterance.get('text', '')
                data.append({
                    'info_id': info_id,
                    'keyword': keyword,
                    'speaker': speaker,
                    'text': text
                })

# 수집한 데이터를 DataFrame으로 저장
df = pd.DataFrame(data, columns=['info_id', 'keyword', 'speaker', 'text'])

# DataFrame을 CSV 파일로 저장
df.to_csv(f'{base_dir}_jeju_dialogues.csv', index=False, encoding='utf-8-sig')

# "jeju_json_lt" 리스트에 저장된 파일들을 확인 (테스트용)
print("Files with valid data:")
print(jeju_json_lt)


Files with valid data:
['L006303.json', 'L013524.json', 'L013538.json', 'L013825.json', 'L014128.json', 'L014362.json', 'L019082.json', 'L019391.json', 'L021963.json', 'L032465.json', 'L046083.json', 'L049336.json', 'L049655.json', 'L049713.json', 'L050634.json', 'L051134.json', 'L072675.json', 'L073721.json', 'L094421.json', 'L095037.json', 'L095210.json', 'L096990.json', 'L098680.json', 'L098719.json', 'L099373.json', 'L099725.json', 'L100490.json', 'L101509.json', 'L181027.json', 'L182114.json', 'L182116.json', 'L190699.json', 'L190860.json', 'L190865.json', 'L191092.json', 'L203450.json', 'L203653.json', 'L206944.json', 'L207395.json', 'L216919.json', 'L244341.json', 'L244343.json', 'L244619.json', 'L244621.json', 'L244881.json', 'L245007.json', 'L245152.json', 'L246092.json', 'L246184.json', 'L246287.json', 'L273192.json', 'L274274.json', 'L275003.json', 'L275056.json', 'L276430.json', 'L276473.json', 'L282658.json']


In [ ]:
len(jeju_json_lt)

57

In [ ]:
df

,info_id,keyword,speaker,text
0,006303,제주시티투어 야간코스,speakerA,나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네..
1,006303,제주시티투어 야간코스,speakerB,생각 그렇게 했을때 실행을 해!! 말만 하지 말고~
2,006303,제주시티투어 야간코스,speakerA,여행 가고 싶은데 멀리는 못가겠고... 어디 가면 좋을까??
3,006303,제주시티투어 야간코스,speakerB,그럴땐 무조건 제주도지!! 뭘 고민해!!
4,006303,제주시티투어 야간코스,speakerA,그럴까?? 그럼 어디가서 뭐 할지 계획 잡아봐야겠당
...,...,...,...,...
911,282658,제주시티투어 야간코스,speakerB,버스는 11월 11일까지 금토에 오후 6시부터 9시까지 1시간 간격으로 돌아간대 ㅋㅋ.
912,282658,제주시티투어 야간코스,speakerA,그럼 마지막으로 하나만 더 물어볼게. 티켓 가격은 얼마고 미리 예매할 수 있어?
913,282658,제주시티투어 야간코스,speakerB,"성인 기준 8천 원이고, '탐나오' 온라인 마켓에서 미리 사거나 현장에서 살 수 있..."
914,282658,제주시티투어 야간코스,speakerA,"생각만 해도 신나는 걸? 좋은 정보 고마워~ 제주도에서 선물 많이 사다 줄게, 기대해!"


In [ ]:
df["keyword"].unique()

array(['제주시티투어 야간코스', '제주시 건입동', '제주항공', '제주도', '제주여행', '제주도, 중국 단체 관광객',
       '제주민속촌', '제주도, 추석 연휴, 관광객', '제주', '제주 월정리', '제주밭한끼', '무민랜드 제주',
       '제주 신화월드', '제주관광공사', '강릉, 양양, 제주, 일본', '제주관광', '제주 드림타워 복합리조트',
       '제주시티투어', '제주901', '제주, WE호텔', '제주시 애월읍 구엄리 돌염전', '제주 수월봉 화산쇄설층',
       '빛의 벙커, 제주, 여름', '중국 단체관광, 제주 드림타워 복합리조트, 매출 증가세',
       '중국 단체관광, 제주 드림타워 복합리조트', '제주 드림타워', '제주, 여름'], dtype=object)

In [ ]:
df[0:30]

,info_id,keyword,speaker,text
0,006303,제주시티투어 야간코스,speakerA,나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네..
1,006303,제주시티투어 야간코스,speakerB,생각 그렇게 했을때 실행을 해!! 말만 하지 말고~
2,006303,제주시티투어 야간코스,speakerA,여행 가고 싶은데 멀리는 못가겠고... 어디 가면 좋을까??
3,006303,제주시티투어 야간코스,speakerB,그럴땐 무조건 제주도지!! 뭘 고민해!!
4,006303,제주시티투어 야간코스,speakerA,그럴까?? 그럼 어디가서 뭐 할지 계획 잡아봐야겠당
5,006303,제주시티투어 야간코스,speakerB,워낙 유명한 관광지니까 어디라도 좋을거야. 제주시티투어 버스같은건 어때?? 난 밤에...
6,006303,제주시티투어 야간코스,speakerA,야~!! 너무 좋다!! 코스가 많아??
7,006303,제주시티투어 야간코스,speakerB,해안 코스랑 도심 코스 2가지 있으니까 너 좋은거 골라봐~
8,006303,제주시티투어 야간코스,speakerA,낮에는 바닷가 갈테니까 도심 코스가 좋겠다~ㅎㅎ. 운행 시간은??
9,006303,제주시티투어 야간코스,speakerB,저녁 6시부터 9시까지 운행한대요~


In [ ]:
df[7270:]

,info_id,keyword,speaker,text


In [ ]:
# 'jeju_dialogues.csv' 파일을 읽어와 데이터프레임 생성
df = pd.read_csv(f'{base_dir}_jeju_dialogues.csv')

# 결과를 저장할 리스트 초기화
result_data = []

# 현재 처리 중인 info_id, keyword 및 history 초기화
current_info_id = None
current_keyword = None
history = ""

# 데이터프레임을 순회하면서 history-response pairs 구성
for index, row in df.iterrows():
    info_id = row['info_id']
    keyword = row['keyword']
    speaker = row['speaker']
    text = row['text']

    # 새로운 info_id로 넘어가면 history를 초기화
    if info_id != current_info_id:
        current_info_id = info_id
        current_keyword = keyword
        history = ""

    # speaker가 speakerA인 경우 text를 history에 누적
    if speaker == "speakerA":
        if history:  # 기존 history가 있으면 공백을 추가하여 구분
            history += " <U> " + text
        else:
            history = "<U> " + text # 첫 발화

    # speaker가 speakerB인 경우 history-response pair를 생성
    elif speaker == "speakerB":
        # 현재 상태의 history와 함께 데이터 저장
        result_data.append({
            'info_id': info_id,
            'keyword': keyword,
            'speaker': speaker,
            'history': history + " <S> ",
            'text': text
        })
        # history를 현재 speakerB의 발화로 초기화
        history += " <S> " + text

# 최종 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(result_data, columns=['info_id', 'keyword', 'speaker', 'history', 'text'])

# 결과를 CSV 파일로 저장
result_df.to_csv(f'{base_dir}_jeju_history_response_pairs.csv', index=False, encoding='utf-8-sig')


In [ ]:
result_df

,info_id,keyword,speaker,history,text
0,6303,제주시티투어 야간코스,speakerB,<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S>,생각 그렇게 했을때 실행을 해!! 말만 하지 말고~
1,6303,제주시티투어 야간코스,speakerB,<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S> 생각...,그럴땐 무조건 제주도지!! 뭘 고민해!!
2,6303,제주시티투어 야간코스,speakerB,<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S> 생각...,워낙 유명한 관광지니까 어디라도 좋을거야. 제주시티투어 버스같은건 어때?? 난 밤에...
3,6303,제주시티투어 야간코스,speakerB,<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S> 생각...,해안 코스랑 도심 코스 2가지 있으니까 너 좋은거 골라봐~
4,6303,제주시티투어 야간코스,speakerB,<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S> 생각...,저녁 6시부터 9시까지 운행한대요~
...,...,...,...,...,...
453,282658,제주시티투어 야간코스,speakerB,<U> 이번 주말에 혼자 제주도 한번 다녀올까 생각 중이야 ㅎㅎ. <S> 진짜? 얼...,버스 타고 제주 전역을 돌면서 야경을 감상하는 코스야!
454,282658,제주시티투어 야간코스,speakerB,<U> 이번 주말에 혼자 제주도 한번 다녀올까 생각 중이야 ㅎㅎ. <S> 진짜? 얼...,"우선 제주국제공항에서 출발해서 무지개 다리로 유명한 도두봉, 어영공원, 동문시장 야..."
455,282658,제주시티투어 야간코스,speakerB,<U> 이번 주말에 혼자 제주도 한번 다녀올까 생각 중이야 ㅎㅎ. <S> 진짜? 얼...,버스는 11월 11일까지 금토에 오후 6시부터 9시까지 1시간 간격으로 돌아간대 ㅋㅋ.
456,282658,제주시티투어 야간코스,speakerB,<U> 이번 주말에 혼자 제주도 한번 다녀올까 생각 중이야 ㅎㅎ. <S> 진짜? 얼...,"성인 기준 8천 원이고, '탐나오' 온라인 마켓에서 미리 사거나 현장에서 살 수 있..."


In [ ]:
result_df['history'][0]

'<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S> '

In [ ]:
result_df['history'][3]

'<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S> 생각 그렇게 했을때 실행을 해!! 말만 하지 말고~ <U> 여행 가고 싶은데 멀리는 못가겠고... 어디 가면 좋을까?? <S> 그럴땐 무조건 제주도지!! 뭘 고민해!! <U> 그럴까?? 그럼 어디가서 뭐 할지 계획 잡아봐야겠당 <S> 워낙 유명한 관광지니까 어디라도 좋을거야. 제주시티투어 버스같은건 어때?? 난 밤에 운행하는 야밤버스 추천함~ <U> 야~!! 너무 좋다!! 코스가 많아?? <S> '

In [ ]:
result_df['history'][5]

'<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S> 생각 그렇게 했을때 실행을 해!! 말만 하지 말고~ <U> 여행 가고 싶은데 멀리는 못가겠고... 어디 가면 좋을까?? <S> 그럴땐 무조건 제주도지!! 뭘 고민해!! <U> 그럴까?? 그럼 어디가서 뭐 할지 계획 잡아봐야겠당 <S> 워낙 유명한 관광지니까 어디라도 좋을거야. 제주시티투어 버스같은건 어때?? 난 밤에 운행하는 야밤버스 추천함~ <U> 야~!! 너무 좋다!! 코스가 많아?? <S> 해안 코스랑 도심 코스 2가지 있으니까 너 좋은거 골라봐~ <U> 낮에는 바닷가 갈테니까 도심 코스가 좋겠다~ㅎㅎ. 운행 시간은?? <S> 저녁 6시부터 9시까지 운행한대요~ <U> 아주 늦게까지는 안하는구나~. 자주 다니는거야?? <S> '

In [ ]:
result_df[:15]

,info_id,keyword,speaker,history,text
0,6303,제주시티투어 야간코스,speakerB,<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S>,생각 그렇게 했을때 실행을 해!! 말만 하지 말고~
1,6303,제주시티투어 야간코스,speakerB,<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S> 생각...,그럴땐 무조건 제주도지!! 뭘 고민해!!
2,6303,제주시티투어 야간코스,speakerB,<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S> 생각...,워낙 유명한 관광지니까 어디라도 좋을거야. 제주시티투어 버스같은건 어때?? 난 밤에...
3,6303,제주시티투어 야간코스,speakerB,<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S> 생각...,해안 코스랑 도심 코스 2가지 있으니까 너 좋은거 골라봐~
4,6303,제주시티투어 야간코스,speakerB,<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S> 생각...,저녁 6시부터 9시까지 운행한대요~
5,6303,제주시티투어 야간코스,speakerB,<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S> 생각...,3회 운행하고 60분 코스~
6,6303,제주시티투어 야간코스,speakerB,<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S> 생각...,탐나오라는 사이트에서만 예약 가능해. 사전 예약 가능하니까 하고 가면 편하지~
7,6303,제주시티투어 야간코스,speakerB,<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S> 생각...,"1회 이용권은 5,000원이고 1일 이용권은 8,000원. 성인 기준이다~"
8,6303,제주시티투어 야간코스,speakerB,<U> 나 워라벨도 생각하면서 살래.. 너무 정신 없이 사는것 같네.. <S> 생각...,말만 하지 말고 진짜로 떠나기나 해라~ㅎㅎ
9,13524,제주시 건입동,speakerB,<U> 오늘은 제주시 건입동에 가볼까? <S>,건입동? 그곳에는 뭐가 있어?


In [ ]:
average_rows_per_group = result_df.groupby("info_id").size().mean()
print(average_rows_per_group)


8.035087719298245


In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # "info_id"의 고유값을 가져옴
# info_ids = result_df["info_id"].unique()

# # train:valid = 9:1 비율로 랜덤하게 분할 (random_state=42 고정)
# train_ids, valid_ids = train_test_split(info_ids, test_size=0.1, random_state=42)

# # train, valid 데이터셋 생성
# train_df = result_df[result_df["info_id"].isin(train_ids)]
# valid_df = result_df[result_df["info_id"].isin(valid_ids)]

# # 결과 확인 (필요 시 출력)
# print(f"Train set size: {train_df.shape[0]} rows")
# print(f"Valid set size: {valid_df.shape[0]} rows")

In [ ]:
# train_df.to_csv('train_jeju_history_response_pairs.csv', index=False, encoding='utf-8-sig')
# valid_df.to_csv('valid_jeju_history_response_pairs.csv', index=False, encoding='utf-8-sig')